# Employee Scheduling

In the next example, a hospital supervisor needs to create a schedule for four nurses over a three-day period, subject to the following conditions:

- Each day is divided into three 8-hour shifts.
- Every day, each shift is assigned to a single nurse, and no nurse works more than one shift.
- Each nurse is assigned to at least two shifts during the three-day period.

https://developers.google.com/optimization/scheduling/employee_scheduling

In [247]:
# Install from master https://github.com/google/or-tools/issues/3094
# !pip install git+https://github.com/google/or-tools.git
from ortools.sat.python import cp_model

In [254]:
num_shifts = 3
num_days = 3
num_nurses = 4

days = range(num_days)
nurses = range(num_nurses)
shifts = range(num_shifts)

In [292]:
model = cp_model.CpModel()

schedules = {}

for day in days:
    for shift in shifts:
        for nurse in nurses:
            schedules[(day, shift, nurse)] = model.NewBoolVar(
                f"day:{day}_shift:{shift}_nurse:{nurse}"
            )

In [293]:
for day in days:
    # Each shift is assigned to a single nurse.
    for shift in shifts:
        model.Add(sum([schedules[(day, shift, nurse)] for nurse in nurses]) == 1)

    # No nurse works more than one shift.
    for nurse in nurses:
        model.Add(sum([schedules[(day, shift, nurse)] for shift in shifts]) <= 1)

In [294]:
min_shifts_per_nurse = (num_shifts * num_days) // num_nurses
if num_shifts * num_days % num_nurses == 0:
    max_shifts_per_nurse = min_shifts_per_nurse
else:
    max_shifts_per_nurse = min_shifts_per_nurse + 1
print(min_shifts_per_nurse, max_shifts_per_nurse)

for nurse in nurses:
    worked_shifts = []

    for day in days:
        worked_shifts.extend(schedules[(day, shift, nurse)] for shift in shifts)

    # Each nurse is assigned to at least two shifts during the three-day period.
    model.Add(sum(worked_shifts) >= min_shifts_per_nurse)
    model.Add(sum(worked_shifts) <= max_shifts_per_nurse)

2 3


In [295]:
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
solver.Solve(
    model
), f"FEASIBLE={cp_model.FEASIBLE}", f"INFEASIBLE={cp_model.INFEASIBLE}", f"OPTIMAL={cp_model.OPTIMAL}"

(4, 'FEASIBLE=2', 'INFEASIBLE=3', 'OPTIMAL=4')

In [296]:
for day in days:
    print("Day", day + 1)
    for shift in shifts:
        is_working = False
        for nurse in nurses:
            if solver.Value(schedules[(day, shift, nurse)]):
                is_working = True
                print(f"Nurse {nurse} working shift {shift}")
        if not is_working:
            print(f"Nurse {nurse} is not working shift {shift}")
    print("")

Day 1
Nurse 3 working shift 0
Nurse 2 working shift 1
Nurse 1 working shift 2

Day 2
Nurse 0 working shift 0
Nurse 3 working shift 1
Nurse 2 working shift 2

Day 3
Nurse 0 working shift 0
Nurse 1 working shift 1
Nurse 3 working shift 2

